In [6]:
import deeplabcut
video_path = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4"
superanimal_name = "superanimal_quadruped"

deeplabcut.video_inference_superanimal([video_path],
                                        superanimal_name,
                                        model_name="resnet_50",
                                        detector_name="fasterrcnn_mobilenet_v3_large_fpn",
                                        video_adapt = False)

Running video inference on ['/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4'] with superanimal_quadruped_resnet_50
Using pytorch for model resnet_50
Loading.... superanimal_quadruped_fasterrcnn_mobilenet_v3_large_fpn


(…)ped_fasterrcnn_mobilenet_v3_large_fpn.pt:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /nfshomes/hwahed/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21.1M/21.1M [00:00<00:00, 227MB/s]


Processing video /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4
Starting to analyze /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4
Video metadata: 
  Overall # of frames:    86
  Duration of video [s]:  2.87
  fps:                    30.0
  resolution:             w=854, h=480

Running detector with batch size 1


...

/fs/nexus-scratch/hwahed/miniconda3/envs/DEEPLABCUT/lib/python3.10/site-packages/deeplabcut/utils/make_labeled_video.py:146: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Dataframe.groupby(level="individuals", axis=1).size().values // 3


Duration of video [s]: 2.87, recorded with 30.0 fps!
Overall # of frames: 86 with cropped frame dimensions: 854 480
Generating frames and creating video.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [00:00<00:00, 86.28it/s]

Video with predictions was saved as /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo


{'/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4': scorer      superanimal_quadruped_fasterrcnn_mobilenet_v3_large_fpn_resnet_50  \
 individuals                                                           animal0   
 bodyparts                                                                nose   
 coords                                                                      x   
 0                                                   529.110779                  
 1                                                   515.281067                  
 2                                                   510.097748                  
 3                                                   507.241669                  
 4                                                   503.147949                  
 ..                                                         ...                  
 81                                                  458.590668                  
 82                            

In [1]:
import pandas as pd
import os

def process_h5_to_text(input_h5_path, output_txt_path, h5_key=None):
    """
    Reads data from a DeepLabCut H5 file and saves it to a text file (CSV).

    Args:
        input_h5_path (str): The full path to the input .h5 file.
        output_txt_path (str): The full path where the output .txt (CSV) file will be saved.
        h5_key (str, optional): The specific key to read from the HDF5 file.
                                 If None, pandas will try to read the first object.
                                 Common DeepLabCut keys include 'df_with_missing'
                                 or the scorer name.
    """
    print(f"Starting processing for H5 file: {input_h5_path}")

    # Check if input file exists
    if not os.path.exists(input_h5_path):
        print(f"Error: Input H5 file not found at '{input_h5_path}'")
        return

    try:
        # Read the HDF5 file
        print(f"Attempting to read H5 file. Key: {'Default' if h5_key is None else h5_key}")
        if h5_key:
            data_frame = pd.read_hdf(input_h5_path, key=h5_key)
        else:
            # Try reading without a key first (works if only one dataset or a default)
            data_frame = pd.read_hdf(input_h5_path)
        
        print("Successfully loaded data from H5 file.")
        print("DataFrame head:\n", data_frame.head())

        # Save the DataFrame to a CSV file
        # CSV is a good, widely compatible text format for tabular data
        data_frame.to_csv(output_txt_path, sep=',', index=True) # index=True to include the frame numbers/timestamps
        print(f"Data successfully saved to: {output_txt_path}")

    except FileNotFoundError:
        print(f"Error: Input H5 file not found at '{input_h5_path}' during read operation.")
    except KeyError as e:
        print(f"Error: Could not find the specified key '{h5_key if h5_key else 'default'}' in the H5 file.")
        print(f"Specific error: {e}")
        print("You might need to inspect the H5 file to find the correct key.")
        print("You can use tools like HDFView or the following Python snippet with h5py:")
        print("import h5py")
        print(f"with h5py.File('{input_h5_path}', 'r') as hf:")
        print(f"    print(list(hf.keys()))")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print(f"Type of error: {type(e)}")

input_file = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid_superanimal_quadruped_fasterrcnn_mobilenet_v3_large_fpn_resnet_50.h5"  

# 2. SET YOUR DESIRED OUTPUT TEXT FILE PATH HERE
#    It's good practice to use a .csv extension for comma-separated values
output_file = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/output.csv"     

# 3. (OPTIONAL) SET THE H5 KEY IF YOU KNOW IT, OTHERWISE LEAVE AS None
#    Common keys: 'df_with_missing', or the full scorer name like
#    'DLC_resnet50_yourProjectName_Date_shuffleX_iterationY'
data_key = None  # Or, for example: 'df_with_missing'
# data_key = 'df_with_missing'
# --- End Configuration ---

process_h5_to_text(input_file, output_file, h5_key=data_key)

Starting processing for H5 file: /fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid_superanimal_quadruped_fasterrcnn_mobilenet_v3_large_fpn_resnet_50.h5
Attempting to read H5 file. Key: Default
Successfully loaded data from H5 file.
DataFrame head:
 scorer      superanimal_quadruped_fasterrcnn_mobilenet_v3_large_fpn_resnet_50  \
individuals                                                           animal0   
bodyparts                                                                nose   
coords                                                                      x   
0                                                   529.110779                  
1                                                   515.281067                  
2                                                   510.097748                  
3                                                   507.241669                  
4                                                   503.147949                  

scorer     

In [49]:
import cv2
import numpy as np
import matplotlib.pyplot as plt # Used for generating a colormap

# --- 1. User Configuration ---
# Update these paths to your video and CSV file
video_path = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/horseVid.mp4"
csv_path = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/testVideo/output.csv" 

# Set the minimum confidence threshold for displaying keypoints
confidence_threshold = 0.6  # Adjust as needed (0.0 to 1.0)

# To find out all available bodyparts in your CSV, you can run these lines once:
# temp_df_for_bodyparts = pd.read_csv(csv_path, header=[0,1,2,3], index_col=0)
# all_bodyparts_in_csv = temp_df_for_bodyparts.columns.get_level_values('bodyparts').unique().tolist()
# print("Available bodyparts in your CSV:", all_bodyparts_in_csv)
# print("Copy and paste the desired bodyparts into the 'bodyparts_to_visualize' list below.")

# Define which bodyparts you want to visualize.
# Example: ['nose', 'left_eye', 'right_eye', 'tail_base']
# Use the printout from above to select from your specific model's bodyparts.
bodyparts_to_visualize = [
    'front_left_paw', 'front_right_paw', 'back_left_paw', 'back_right_paw',
    'front_left_knee', 'front_right_knee', 'back_left_knee', 'back_right_knee',
    'front_right_thai', 'front_left_thai', 'back_right_thai', 'back_left_thai',
    'nose', 'upper_jaw', 'lower_jaw', 'mouth_end_right', 'mouth_end_left',
    'right_eye', 'left_eye', 'right_earbase', 'left_earbase', 'neck_base', 'neck_end',
    'throat_end', 'back_base', 'back_end'
    # Add or remove bodyparts from the "superanimal_quadruped" model as needed
]

# bodyparts_to_visualize = [
#     'nose', 'right_eye', 'left_eye', 'right_earbase', 'left_earbase',
#     'neck_base', 'back_base', 'back_middle', 'back_end', 'tail_base', 'tail_end',
#     'front_left_paw', 'front_right_paw', 'back_left_paw', 'back_right_paw',
#     'front_left_knee', 'front_right_knee', 'back_left_knee', 'back_right_knee',
#     # Add or remove bodyparts from the "superanimal_quadruped" model as needed
# ]
# --- 2. Load Data ---
try:
    # The header in DeepLabCut CSVs for multiple animals typically has 4 levels:
    # Level 0: scorer
    # Level 1: individuals (e.g., animal0, animal1)
    # Level 2: bodyparts (e.g., nose, left_eye)
    # Level 3: coords (x, y, likelihood)
    # The first column (index_col=0) contains the frame numbers.
    df = pd.read_csv(csv_path, header=[0, 1, 2, 3], index_col=0)
except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_path}")
    exit()
except Exception as e:
    print(f"Error loading CSV: {e}")
    print("Please ensure the CSV path is correct and the file is a standard DeepLabCut multi-animal output CSV.")
    exit()

In [47]:
# Extract necessary information from the DataFrame columns
try:
    scorer_name = df.columns.get_level_values('scorer').unique()[0]
    individuals = df.columns.get_level_values('individuals').unique().tolist()
    all_bodyparts_from_df = df.columns.get_level_values('bodyparts').unique().tolist()
except KeyError as e:
    print(f"Error: CSV header does not match expected DeepLabCut multi-animal format. Missing level: {e}")
    print("Expected header levels: 'scorer', 'individuals', 'bodyparts', 'coords'")
    exit()

print(f"Scorer: {scorer_name}")
print(f"Individuals: {individuals}")
print(f"Available bodyparts in CSV: {all_bodyparts_from_df}")

# Filter bodyparts_to_visualize to only include those present in the CSV
original_bodyparts_to_visualize = list(bodyparts_to_visualize)
bodyparts_to_visualize = [bp for bp in bodyparts_to_visualize if bp in all_bodyparts_from_df]

if not bodyparts_to_visualize:
    if original_bodyparts_to_visualize: # If user specified some, but none matched
         print(f"Warning: None of the bodyparts in your 'bodyparts_to_visualize' list ({original_bodyparts_to_visualize}) were found in the CSV.")
    print(f"Defaulting to display ALL available bodyparts from the CSV: {all_bodyparts_from_df}")
    bodyparts_to_visualize = all_bodyparts_from_df
    if not bodyparts_to_visualize:
        print("Error: No bodyparts could be identified from the CSV header.")
        exit()

# --- 3. Video and Visualization Setup ---


Scorer: superanimal_quadruped_fasterrcnn_mobilenet_v3_large_fpn_resnet_50
Individuals: ['animal0', 'animal1', 'animal2', 'animal3', 'animal4', 'animal5', 'animal6', 'animal7', 'animal8', 'animal9']
Available bodyparts in CSV: ['nose', 'upper_jaw', 'lower_jaw', 'mouth_end_right', 'mouth_end_left', 'right_eye', 'right_earbase', 'right_earend', 'right_antler_base', 'right_antler_end', 'left_eye', 'left_earbase', 'left_earend', 'left_antler_base', 'left_antler_end', 'neck_base', 'neck_end', 'throat_base', 'throat_end', 'back_base', 'back_end', 'back_middle', 'tail_base', 'tail_end', 'front_left_thai', 'front_left_knee', 'front_left_paw', 'front_right_thai', 'front_right_knee', 'front_right_paw', 'back_left_paw', 'back_left_thai', 'back_right_thai', 'back_left_knee', 'back_right_knee', 'back_right_paw', 'belly_bottom', 'body_middle_right', 'body_middle_left']


In [48]:
cap = cv2.VideoCapture(video_path)
process_all_frames = False
start_frame_index = 0      # 0-based index of the first frame to process
end_frame_index = 200      # Process frames up to (but not including) this index. Set higher to process more.
output_video_filename = "/fs/nexus-scratch/hwahed/dlcDatasetMaker/output.mp4"
output_video_fourcc = cv2.VideoWriter_fourcc(*'mp4v')
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0: # Handle case where FPS might not be readable
    print("Warning: Could not read FPS from input video. Defaulting to 25 FPS for output.")
    fps = 25.0
    
total_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
max_df_frame_idx = df.index.max() if not df.empty else -1
num_frames_available_for_processing = min(total_video_frames, max_df_frame_idx + 1)

if process_all_frames:
    frames_to_iterate = range(num_frames_available_for_processing)
else:
    actual_start_frame = max(0, start_frame_index)
    actual_end_frame = min(num_frames_available_for_processing, end_frame_index)
    frames_to_iterate = range(actual_start_frame, actual_end_frame)
    if actual_start_frame >= actual_end_frame:
        print(f"Warning: Start frame ({actual_start_frame}) is not before end frame ({actual_end_frame}). No frames will be processed.")
        frames_to_iterate = range(0)

# Initialize VideoWriter
video_writer = cv2.VideoWriter(output_video_filename, output_video_fourcc, fps, (frame_width, frame_height))
if not video_writer.isOpened():
    print(f"Error: Could not open VideoWriter for {output_video_filename}. Check FourCC code or permissions.")
    cap.release()
    exit()

# Generate a color map for different bodyparts
cmap = plt.cm.get_cmap('tab20', len(bodyparts_to_visualize))
bodypart_colors = {}
for i, bp in enumerate(bodyparts_to_visualize):
    color_rgba = cmap(i / max(1, len(bodyparts_to_visualize)-1) if len(bodyparts_to_visualize) > 1 else 0.5 )
    bodypart_colors[bp] = tuple(int(c * 255) for c in color_rgba[:3])

# --- 4. Process Video and Write to Output Video ---
print(f"\nProcessing frames from {min(frames_to_iterate, default=0)+1} to {max(frames_to_iterate, default=-1)+1}...")
print(f"Annotated video will be saved as '{output_video_filename}'")
print(f"Displaying keypoints with confidence >= {confidence_threshold}")
print(f"Selected bodyparts for visualization: {bodyparts_to_visualize}")

processed_frame_count = 0
for frame_idx in frames_to_iterate:
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()

    if not ret:
        print(f"Warning: Could not read frame {frame_idx} from video. Skipping.")
        continue

    annotated_frame = frame.copy()

    if frame_idx in df.index:
        frame_data_series = df.loc[frame_idx]

        for individual_id in individuals:
            for bp in bodyparts_to_visualize:
                try:
                    x = frame_data_series.loc[(scorer_name, individual_id, bp, 'x')]
                    y = frame_data_series.loc[(scorer_name, individual_id, bp, 'y')]
                    likelihood = frame_data_series.loc[(scorer_name, individual_id, bp, 'likelihood')]

                    if pd.isna(x) or pd.isna(y) or pd.isna(likelihood) or likelihood < confidence_threshold:
                        continue
                    
                    x_coord, y_coord = int(x), int(y)

                    if x_coord >= 0 and y_coord >= 0:
                        color = bodypart_colors.get(bp, (0, 0, 255))
                        cv2.circle(annotated_frame, (x_coord, y_coord), 5, color, -1)
                        label = f"{bp[:3]}_{individual_id.replace('animal', '')} ({likelihood:.2f})"
                        cv2.putText(annotated_frame, label, (x_coord + 7, y_coord - 7),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1, cv2.LINE_AA)
                except KeyError:
                    pass
                except (ValueError, TypeError):
                    pass

    # Add frame number text to the image
    cv2.putText(annotated_frame, f"Frame: {frame_idx + 1}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 3, cv2.LINE_AA)
    cv2.putText(annotated_frame, f"Frame: {frame_idx + 1}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)
    
    cv2.putText(annotated_frame, f"Threshold: {confidence_threshold:.2f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 3, cv2.LINE_AA)
    cv2.putText(annotated_frame, f"Threshold: {confidence_threshold:.2f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1, cv2.LINE_AA)

    # Write the frame to the output video
    video_writer.write(annotated_frame)
    processed_frame_count += 1

    if processed_frame_count % 50 == 0: # Print progress update every 50 frames
        print(f"Processed and written frame {frame_idx + 1} to video...")

# --- 5. Cleanup ---
cap.release()
video_writer.release() # Release the VideoWriter object
print(f"\nFinished processing. {processed_frame_count} annotated frames written to '{output_video_filename}'.")

/tmp/ipykernel_270831/2476603151.py:40: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('tab20', len(bodyparts_to_visualize))



Processing frames from 1 to 86...
Annotated video will be saved as '/fs/nexus-scratch/hwahed/dlcDatasetMaker/output.mp4'
Displaying keypoints with confidence >= 0.6
Selected bodyparts for visualization: ['front_left_paw', 'front_right_paw', 'back_left_paw', 'back_right_paw', 'front_left_knee', 'front_right_knee', 'back_left_knee', 'back_right_knee', 'front_right_thai', 'front_left_thai', 'back_right_thai', 'back_left_thai', 'nose', 'upper_jaw', 'lower_jaw', 'mouth_end_right', 'mouth_end_left', 'right_eye', 'left_eye', 'right_earbase', 'left_earbase', 'neck_base', 'neck_end', 'throat_end', 'back_base', 'back_middle', 'back_end']
Processed and written frame 50 to video...

Finished processing. 86 annotated frames written to '/fs/nexus-scratch/hwahed/dlcDatasetMaker/output.mp4'.
